In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
def get_pad_layer(pad_type):
    if(pad_type in ['refl','reflect']):
        PadLayer = nn.ReflectionPad2d
    elif(pad_type in ['repl','replicate']):
        PadLayer = nn.ReplicationPad2d
    elif(pad_type=='zero'):
        PadLayer = nn.ZeroPad2d
    else:
        print('Pad type [%s] not recognized'%pad_type)
    return PadLayer

class BlurPool(nn.Module):
    def __init__(self, pad_type='reflect', filter_size=2, stride=2, channels=None, pad_off=0):
        super(BlurPool, self).__init__()
        self.filter_size = filter_size
        self.pad_off = pad_off
        self.pad_sizes = [int(1.*(filter_size-1)/2), int(np.ceil(1.*(filter_size-1)/2)), int(1.*(filter_size-1)/2), int(np.ceil(1.*(filter_size-1)/2))]
        self.channels = channels
        self.pad_sizes = [pad_size+pad_off for pad_size in self.pad_sizes]
        self.stride = stride
        self.off = int((self.stride-1)/2.)
        
        if(self.filter_size==1):
            a = np.array([1.,])
        elif(self.filter_size==2):
            a = np.array([1., 1.])
        elif(self.filter_size==3):
            a = np.array([1., 2., 1.])
        elif(self.filter_size==4):    
            a = np.array([1., 3., 3., 1.])
        elif(self.filter_size==5):    
            a = np.array([1., 4., 6., 4., 1.])
        elif(self.filter_size==6):    
            a = np.array([1., 5., 10., 10., 5., 1.])
        elif(self.filter_size==7):    
            a = np.array([1., 6., 15., 20., 15., 6., 1.])
        
        filt = torch.Tensor(a[:,None]*a[None,:])
        filt = filt/torch.sum(filt)
        self.register_buffer('filt', filt[None,None,:,:].repeat((self.channels,1,1,1)))
        
        self.pad = get_pad_layer(pad_type)(self.pad_sizes)
    
    def forward(self, input):
        if(self.filter_size==1):
            if(self.pad_off==0):
                return input[:,:,::self.stride,::self.stride]    
            else:
                return self.pad(input)[:,:,::self.stride,::self.stride]
        else:
            return F.conv2d(self.pad(inp), self.filt, stride=self.stride, groups=inp.shape[1])
        

In [4]:
class AlexNetNMP(nn.Module):

    def __init__(self, num_classes=1000, filter_size=1):
        super(AlexNetNMP, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            BlurPool(filter_size=filter_size, stride=2, channels=64, pad_off=-1),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            BlurPool(filter_size=filter_size, stride=2, channels=192, pad_off=-1),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            BlurPool(filter_size=filter_size, stride=2, channels=256, pad_off=-1),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        # embed()
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [5]:
class NetNMP(nn.Module):
    def __init__(self, filter_size=1):
        super(NetNMP, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(inplace=True),
            BlurPool(filter_size=filter_size, stride=2, channels=6),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(inplace=True),
            BlurPool(filter_size=filter_size, stride=2, channels=16),
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, 10),
        )
        

    def forward(self, x):
        x = self.features(x)
        x = x.reshape(x.shape[0], 16 * 5 * 5)
        x = self.classifier(x)
        return x

In [6]:
class BestNet(nn.Module):
    def __init__(self, filter_size=1):
        super(BestNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            BlurPool(filter_size=filter_size, stride=2, channels=32),
            nn.Dropout(p=0.25),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            BlurPool(filter_size=filter_size, stride=2, channels=64),
            nn.Dropout(p=0.25),
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 8 * 8, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.25),
            nn.Linear(512, 10),
        )
        

    def forward(self, x):
        x = self.features(x)
        x = x.reshape(x.shape[0], 64*8*8)
        x = self.classifier(x)
        return x

In [7]:
net = BestNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [8]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 2.162
[1,   200] loss: 1.869
[1,   300] loss: 1.760
[1,   400] loss: 1.633
[1,   500] loss: 1.568
[1,   600] loss: 1.511
[1,   700] loss: 1.473
[1,   800] loss: 1.423
[1,   900] loss: 1.393
[1,  1000] loss: 1.398
[1,  1100] loss: 1.390


KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [100]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 79 %
Accuracy of   car : 84 %
Accuracy of  bird : 51 %
Accuracy of   cat : 45 %
Accuracy of  deer : 64 %
Accuracy of   dog : 60 %
Accuracy of  frog : 71 %
Accuracy of horse : 75 %
Accuracy of  ship : 78 %
Accuracy of truck : 80 %
